In [1]:
import pandas as pd
import numpy as np
import pdfplumber as scrapper
import PyPDF2
from io import StringIO
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

## Step 1. Importing all items and products

In [2]:
# Identify all items
item_num = 0
products_list = [] # List to store product names
# item_prod_relations = {} # Dictionary to store relationships between items and products # NOT USED
prod_num_in_items = 0 # Counter for product numbers in items

path = "C:/Users/smvan/CFFS-S23/CFFS-22-23"
os.chdir(path)
filepath = glob.glob(os.path.join(os.getcwd(), "data", "AMS", "*.pdf"))

col_list = ["ItemId", "Description", "Qty", "UOM", "PrepId"]
df = pd.DataFrame(columns=col_list) # Create an empty (df) DataFrame with the col_list elements as the column names
df

for file in filepath:
    
    # Some of the tools I have commented got DeprecationError so I changed them to the latest version

    def generate_items(pdf):
        file = open(str(pdf), "rb") # Open the PDF file in binary read mode
        # pdfReader = PyPDF2.PdfFileReader(file)
        pdfReader = PyPDF2.PdfReader(file) # Create a PDF reader object using the latest version
        # pageObj = pdfReader.getPage(0) 
        pageObj = pdfReader.pages[0]  # Get the first page of the PDF using the latest version
        # myfile = pageObj.extractText()
        myfile = pageObj.extract_text() # Extract text from the PDF page using the latest version

        product = myfile.partition('\n')[0]
        products_list.append(product)      # Add product names in the list
        myfile = myfile.split("\n", 4)[4]
        myfile = myfile.splitlines()
        df = pd.read_csv(StringIO("\n". join(myfile)), sep=" ", names=range(20), on_bad_lines="skip", quotechar=None, quoting=3)

        amounts = df.iloc[:, 0:4]
        amounts_col = ["Single", "Double", "Triple", "Half"]
        amounts.columns = amounts_col
        ingredient = df.iloc[:, 5:]
        ingredient["Description"] = ingredient.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
        ingredient["Description"] = ingredient["Description"].str.replace("nan","")
        ingredient['Description'] = ingredient['Description'].str.extract('(\d*\.*\d*\s*[A-Z].*$)')
        df = pd.concat([amounts, ingredient["Description"]], axis=1)

        df["ItemId"] = ""

        df[["Amount", "Unit"]] = df["Single"].str.extract(pat=r"(\d+\.?\d*)(.*)")
        ind_num = df[df["Amount"].isnull()].index.tolist()
        min_nan = min(ind_num)
        df = df.iloc[:min_nan]
        df["InventoryGroup"] = ""

        global prod_num_in_items

        df["PrepId"] = "P-" + str(prod_num_in_items) # add the product id
        prod_num_in_items += 1                       # increment prod_num_in_items
        df = df.rename(columns={"Amount": "Qty", "Unit":"UOM"})

        for ind, row in df.iterrows():

            global item_num
            df.loc[ind, "ItemId"] = "I-" + str(item_num) # Assign a unique item ID
            item_num += 1

        item_col_list = ["ItemId", "Description", "Qty", "UOM", "PrepId"]
        df = df[item_col_list]

        return df
    
    temp_df = generate_items(file) # Generate items from the file
    global df
    df = pd.concat([df, temp_df], axis=0)  # Concatenate the generated items with the main DataFrame
    
items = df    
print(len(items))
items.head(30)


# My understanding of the code:
# Each recipe ingredients are added onto the dataframe and they are numbered 0 to whatever the number of ingredients there are
# in the recipe and then when we start a new recipe we again start from 0 and go upto the number of items in the recipe

# The items are all listed accordingly to the products .pdf in the AMS data folder. Here the items are all listed starting from 
# the 2022.3 Full Aloo Tikki Burger

154


,ItemId,Description,Qty,UOM,PrepId
0,I-0,BUN Multigrain Vegan 4.5,1.000,ea,P-0
1,I-1,2022.3 Mint Chutney,20.000,g,P-0
2,I-2,Tamarind Chutney 1L,20.000,ml,P-0
3,I-3,Lettuce - Butter,0.050,ea,P-0
4,I-4,ONION RED JUMBO,10.000,g,P-0
5,I-5,TOMATO - 6 X 7 MED,30.000,g,P-0
6,I-6,CUCUMBER LONG ENGLI S,15.000,g,P-0
7,I-7,PATTY PATTY 2022.3 Aloo Tikki,1.000,PATTY,P-0
0,I-8,2022.3 Potato Patty,120.000,g,P-1
1,I-9,Sabutdana,15.000,g,P-1


In [3]:
prep_num = 0

# calculate the product 

def generate_products():
    global products_list # Access the global variable 'products_list'
    columns = ["Description"] # Create a list of column names
    # Create a DataFrame using 'products_list' and column names
    df = pd.DataFrame(products_list, columns=columns) 
    df["PrepId"] = "" # Add an empty column 'PrepId' to the DataFrame
    
    
    for ind, row in df.iterrows(): # Iterate over each row in the DataFrame

        global prep_num
        df.loc[ind, "PrepId"] = "P-" + str(prep_num) # Assign a PrepId to each row using 'prep_num'
        prep_num += 1
    
    df['Description'] = df['Description'].str.replace('^2022.3 ','') # Remove the pattern '^2022.3 ' from the 'Description' column
    df['Description'] = df['Description'].str.replace('\s*FLAVOUR LAB \(FLAVLAB\)','') # Remove the pattern '\s*FLAVOUR LAB \(FLAVLAB\)' from the 'Description' column
    prep_col_list = ["PrepId", "Description"] # Create a list of column names to keep in the final DataFrame
    df = df[prep_col_list] # Select only the columns specified in 'prep_col_list'
    
    return df
    
products = generate_products() # Generate products using the 'generate_products' function
products # Print the generated products DataFrame

,PrepId,Description
0,P-0,Full Aloo Tikki Burger
1,P-1,Aloo Tikki
2,P-2,Confit Garlic Oil
3,P-3,Cooked Bowtie
4,P-4,Double Down Chicken
5,P-5,Fattoush Salad
6,P-6,Fried Chickpea
7,P-7,Full KFC Double Down
8,P-8,Full Smoked Tofu Burger
9,P-9,Gochujang Alfredo


### Step 1-a. Identifying *Pure Items*
**Pure items**: Items that are independent on its own. These items are NOT used as an ingredient for another product. 

In [4]:
# Made some changes below. Since there is no notebooks folder we cannot include it in the path

# pure_df contains items used for only one product.

mask = items["Description"].str.contains('2022.3')
masked = items[mask]

pure_df = pd.merge(items, masked, how='outer', indicator='Exist')

pure_df = pure_df.loc[pure_df['Exist'] != 'both']
pure_df = pure_df.drop(["Exist"], axis=1)
# pure_df.to_csv("notebooks/data/AMS/preprocessed/pure_items.csv", index=False)
pure_df.to_csv("data/AMS/preprocessed/pure_items.csv", index=False)
pure_df

,ItemId,Description,Qty,UOM,PrepId
0,I-0,BUN Multigrain Vegan 4.5,1.000,ea,P-0
2,I-2,Tamarind Chutney 1L,20.000,ml,P-0
3,I-3,Lettuce - Butter,0.050,ea,P-0
4,I-4,ONION RED JUMBO,10.000,g,P-0
5,I-5,TOMATO - 6 X 7 MED,30.000,g,P-0
...,...,...,...,...,...
149,I-149,SALT COARSE KOSHER,10.000,g,P-27
150,I-150,GARLIC WHOLE PEELED,30.000,g,P-27
151,I-151,Juice - Lemon Fresh Sque e,100.000,g,P-27
152,I-152,PFO OIL CANOLA JIB AN T,3000.000,ml,P-28


Filter out **child items** which are used as items for other products. 

In [5]:
# child_items = masked.copy()
# child_items = child_items.reset_index().drop(["index"], axis=1)
# child_items["OriginId"]= ""
# child_items.to_csv("notebooks/data/AMS/preprocessed/Child_parent_df.csv", index=False)

# child_items = pd.read_csv("notebooks/data/AMS/preprocessed/Child_parent_df.csv")
# print(child_items["OriginId"].unique().tolist())
# child_items

- `PrepId`: what that item will be used to make.
- `ProdId`: the made product will be used in future to make another product.

In [6]:
# child_items = child_items.drop("PrepId", axis=1)
# child_items = child_items.rename(columns={"OriginId": "PrepId", "PrepId":"ProdId"})
# child_items.to_csv("notebooks/data/AMS/preprocessed/Child_parent_df.csv", index=False)

# for ind, row in child_items.iterrows():
#     if pd.isnull(child_items.at[ind,'PrepId']) == True:
#         child_items.loc[ind, "PrepId"] = row["ProdId"]
        
# child_items.to_csv("notebooks/data/AMS/preprocessed/Child_parent_df.csv", index=False)

# child_items = pd.read_csv("notebooks/data/AMS/preprocessed/Child_parent_df.csv")


# Made some changes below. Since there is no notebooks folder we cannot include it in the path

child_items = pd.read_csv("data/AMS/preprocessed/Child_parent_df.csv")
child_items

,ItemId,Description,Qty,UOM,ProdId,PrepId
0,I-1,2022.3 Basic Pickling Liqui d,1000.00,g,P-0,P-0
1,I-3,PORT PORT 2022.3 Double Down Chick e,2.00,PORT,P-1,P-20
2,I-4,PATTY PATTY 2022.3 Rice Patty Cake,1.00,PATTY,P-1,P-21
3,I-13,2022.3 Mint Chutney,20.00,g,P-3,P-25
4,I-19,PATTY PATTY 2022.3 Aloo Tikki,1.00,PATTY,P-3,P-18
5,I-47,2022.3 Cooked Penne,10.00,oz,P-9,P-9
6,I-48,2022.3 Marinara,6.00,oz,P-9,P-9
7,I-58,2022.3 Thyme Oil,1000.00,g,P-10,P-27
8,I-81,2022.3 Smoked Paprika Ru,30.00,g,P-15,P-5
9,I-96,2022.3 Potato Patty,120.00,g,P-18,P-12


In [7]:
looped_list = []

for ind, row in child_items.iterrows():
    if row["PrepId"] in ['P-20', 'P-18', 'P-15']:
        looped_list.append(row["ProdId"])

looped_list

['P-1', 'P-3', 'P-24']

In [8]:
parent_prod = ["P-3", "P-1", "P-24", "P-22", "P-13", "P-20"] 

Isolate items that are used for **child products**. 

In [9]:
child_id = child_items.dropna(subset=['PrepId'])
child_list = []

# Identify products that are used as used as ingredients for other products.
for item in child_id["PrepId"].unique().tolist():
    if item in child_items["ProdId"].tolist():
        child_list.append(item)
        
# Identify looped-child products and remove from dataframe. 
col_list = ["ItemId", "Description", "Qty", "UOM", "PrepId"]
new_df = pd.DataFrame(columns=col_list)

for item in child_items["PrepId"].unique().tolist():
    for ind, row in items.iterrows():
        if row["PrepId"] == item:
            new_df = new_df.append(pd.Series(row))

new_df = new_df.reset_index()
drop_df = pd.DataFrame(columns=["ItemId", "Description", "Qty", "UOM", "PrepId"])

# Items used for child products 
for ind, row in new_df.iterrows():
    if row["PrepId"] in child_list:
        new_df = new_df.drop(ind)
        drop_df = drop_df.append(row)
        
new_df = new_df.drop(["index"], axis=1)
print("Removed: ", child_list)
print("Kept: ", new_df["PrepId"].unique().tolist())
new_df

Removed:  ['P-0', 'P-20', 'P-18', 'P-9', 'P-15']
Kept:  ['P-21', 'P-25', 'P-27', 'P-5', 'P-12', 'P-28', 'P-2', 'P-17']


,ItemId,Description,Qty,UOM,PrepId
14,I-118,2022.2 Cooked Sushi Rice,1350.000,g,P-21
15,I-119,SHICHIMI TOGARASHI,15.000,g,P-21
16,I-120,PFO OIL CANOLA JIB AN T,100.000,ml,P-21
17,I-134,Tofu MED Firm Sunrise,36.000,lb,P-25
18,I-135,Hickory Wood Chips,1000.000,g,P-25
19,I-136,SALT COARSE KOSHER,250.000,g,P-25
25,I-147,SPICE SUMAC ground,30.000,g,P-27
26,I-148,Olive Oil Extra virgin,400.000,ml,P-27
27,I-149,SALT COARSE KOSHER,10.000,g,P-27
28,I-150,GARLIC WHOLE PEELED,30.000,g,P-27


In [10]:
for ind, row in drop_df.iterrows():
    if row["PrepId"] in ["P-0", "P-9"]:
        drop_df = drop_df.drop(ind)
        
drop_df
# drop_df.to_csv("notebooks/data/AMS/preprocessed/Looped_child_df.csv", index=False)

,ItemId,Description,Qty,UOM,PrepId,index
8,I-112,POTATO RUSSET BAKER,150.000,g,P-20,0.0
9,I-113,Cheese 3 Blend Shred,50.000,g,P-20,1.0
10,I-114,Bacon Raw 14-22ct econo,2.000,slice,P-20,2.0
11,I-115,CREAM SOUR 4L,30.000,g,P-20,3.0
12,I-116,ONIONS GREEN,5.000,g,P-20,4.0
13,I-117,SPICE CHILI POWDER,1.000,g,P-20,5.0
20,I-101,Radish - Daikon,1000.000,g,P-18,0.0
21,I-102,2022.3 Basic Pickling Liqui d,1000.000,g,P-18,1.0
45,I-90,ONIONS,750.000,g,P-15,0.0
46,I-91,GARLIC WHOLE PEELED,250.000,g,P-15,1.0


In [11]:
# Made some changes below. Since there is no notebooks folder we cannot include it in the path

pure_items = pd.read_csv("data/AMS/preprocessed/Pure_items_labelled.csv")
# pure_items = pd.read_csv("notebooks/data/AMS/preprocessed/Pure_items_labelled.csv")

# Filter out complicated items
for ind, row in pure_items.iterrows():
    if row["ProdId"] in parent_prod:
        pure_items = pure_items.drop(ind)

---

## Step 2. Merge Dataframes

In [12]:
pure_items = pd.read_csv("data/AMS/preprocessed/Labelled_per_Items.csv")
# pure_items = pure_items.drop(["Weight (g)", "Total CO2 Emission (g)", "Total Nitrogen Emission (g)", 
#                               "Total Freshwater Withdrawals (mL)"], axis=1)
pure_items = pure_items.drop("Weight (g)", axis=1)
pure_items = pure_items.rename(columns={"ConvertToQty":"Weight (g)"})
# pure_items = pure_items.rename(columns={"ConvertToQty":"Weight (g)",
#                                        "CO2 Emission (g CO2)": "Total CO2 Emission (g)",
#                                        "Nitrogen Lost (g)": "Total Nitrogen Emission (g)",
#                                        "Water Withdrawals (mL)": "Total Freshwater Withdrawals (mL)"})
pure_items = pure_items[["ConversionId", "PrepId", "Weight (g)", "GHG Emission (g) / 100g", "N lost (g) / 100g", 
                         "Stress-Weighted Water Use (L) / 100g", 
                         "Total CO2 Emission (g)", "Total Nitrogen Emission (g)",
                         "Total Freshwater Withdrawals (mL)"]]
pure_items

,ConversionId,PrepId,Weight (g),GHG Emission (g) / 100g,N lost (g) / 100g,Stress-Weighted Water Use (L) / 100g,Total CO2 Emission (g),Total Nitrogen Emission (g),Total Freshwater Withdrawals (mL)
0,I-0,P-0,1000.00,30.620000,0.790000,0.990000,306.200000,7.900000,9900.000
1,I-2,P-1,1.00,0.000000,0.000000,0.000943,0.000000,0.000000,1.000
2,I-77,P-14,5000.00,0.000000,0.000000,0.070126,0.000000,0.000000,5000.000
3,I-95,P-17,100.00,0.000000,0.000000,0.017544,0.000000,0.000000,100.000
4,I-6,P-1,30.00,19.618868,0.223585,2.179245,20.796000,0.237000,2310.000
...,...,...,...,...,...,...,...,...,...
132,I-22,P-4,35.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000
133,I-50,P-9,1.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000
134,I-74,P-13,1.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000
135,I-43,P-8,24.00,90.752308,1.225846,16.710462,235.956000,3.187200,43447.200


In [13]:
child_prods = pd.read_csv("data/AMS/preprocessed/Labelled_per_Child_Items.csv")
child_prods["ConversionId"] = child_prods["ProdId"]
child_items = child_prods[["ConversionId", "PrepId", "Weight (g)", "GHG Emission (g) / 100g", "N lost (g) / 100g", 
                           'm^2 land use/kg product / 100g', "Stress-Weighted Water Use (L) / 100g", 
                           "Total CO2 Emission (g)", "Total Nitrogen Emission (g)",
                           "Total Freshwater Withdrawals (mL)"]]
child_items

KeyError: "['m^2 land use/kg product / 100g'] not in index"

In [ ]:
df = pd.concat([pure_items, child_items], axis = 0)
df

In [ ]:
final_df = pd.DataFrame(columns=["ProdId", "Weight (g)", "Total CO2 Emission (g)", "Total Nitrogen Emission (g)",
                                 "Total Freshwater Withdrawals (mL)"])

# df = pd.read_csv("data/AMS/preprocessed/processed_items.csv")

all_prod = df["PrepId"].unique().tolist()

for prod in all_prod:
    temp_df = df.loc[df["PrepId"] == prod]
    ProdId = prod
    Weight = 0
    total_CO2 = 0
    total_N = 0
    total_Water = 0
    total_Land = 0
    for ind, row in temp_df.iterrows(): 
        Weight += row["Weight (g)"]
        total_CO2 += row["Total CO2 Emission (g)"]
        total_N += row["Total Nitrogen Emission (g)"]
        total_Water += row["Total Freshwater Withdrawals (mL)"]
        total_Land += row["Total Land"]
        
    info = [ProdId, Weight, total_CO2, total_N, total_Water]
    
    # See what line below is doing
    final_df.loc[len(final_df.index)] = info
    # final_df = info

final_df

In [ ]:
def calculate_100g_emissions(df):
    for ind, row in df.iterrows():
        df.loc[ind, "GHG Emission (g) / 100g"] = (row["Total CO2 Emission (g)"] / row["Weight (g)"]) * 100
        df.loc[ind, "N lost (g) / 100g"] = (row["Total Nitrogen Emission (g)"] / row["Weight (g)"]) * 100
        df.loc[ind, "Stress-Weighted Water Use (L) / 100g"] = ((row["Total Freshwater Withdrawals (mL)"] / 1000) / row["Weight (g)"]) * 100
    return df

calculate_100g_emissions(final_df)

In [ ]:
fifty_cutoff = 180.12
overall_baseline = 360.25

GHG_baseline = 381.13
nitrogen_baseline = 4.21
water_baseline = 1501.2

def create_results_all_factors(df):
    df["Combined Label"] = ""

    def calculate_all_factors(ghg, nitrogen, water):
        weighted_ghg = ghg / (3 * GHG_baseline)
        weighted_nitrogen = nitrogen / (3 * nitrogen_baseline)
        weighted_water = water / (3 * water_baseline)
        return weighted_ghg + weighted_nitrogen + weighted_water

    for ind, row in df.iterrows():
        label = calculate_all_factors(row["GHG Emission (g) / 100g"], row["N lost (g) / 100g"],
                                      row["Stress-Weighted Water Use (L) / 100g"])

        if label <= 0.5:
            df.loc[ind, "Combined Label"] = "Green"
        elif label >= 1:
            df.loc[ind, "Combined Label"] = "Red"
        elif (label > 0.5) and (label < 1):
            df.loc[ind, "Combined Label"] = "Yellow"
    return df

df = create_results_all_factors(final_df)
print(len(df))
df

In [ ]:
looped_menus = ['P-20', 'P-18', 'P-15']
prod_list = ['P-1', 'P-3', 'P-24']

# this is a for loop for rows and we are also using the index (ind)
# So we are dropping menu items: looped_menus from the dataframe. WHY???

for ind, row in df.iterrows():
    if row["ProdId"] in looped_menus:
        df = df.drop(ind)
        
df

In [ ]:
df = df.merge(products, left_on="ProdId", right_on="PrepId")
df = df.drop("PrepId", axis=1)
df_labelled = df[["ProdId", "GHG Emission (g) / 100g", "N lost (g) / 100g", "Stress-Weighted Water Use (L) / 100g",
                 "Combined Label", "Description"]]
df_labelled.to_csv("data/AMS/final/Flavour_Lab_Labelled.csv", index=False)


# The csv file is loading properly but not the Excel file

df_labelled.to_excel("data/AMS/final/Flavour_Lab_Labelled.xlsx")  
df_labelled

---
## Step 3. Summary and Visualizations

In [ ]:
# changed the dataframe labelling so that Combined Label becomes Combined_Label
df_labelled = df_labelled.rename(columns={"Combined Label": "Combined_Label"})
display(df_labelled.Combined_Label.value_counts())

In [ ]:
df_summary = df_labelled["Combined_Label"].value_counts().to_frame("Label Counts")
df_summary["Percentage"] = (df_summary["Label Counts"] / df_summary["Label Counts"].sum()) * 100
df_summary

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(11,6), sharey=False)

palette = {"Green":"tab:green", "Red":"tab:red", "Yellow":"tab:orange"}
labels = ["Green", "Red", "Yellow"]

# Barplot on the right side
ax1 = sns.countplot(data=df_labelled, x="Combined_Label", ax=axes[0], palette=palette)
ax1.set_xlabel("Combined Label")
ax1.set_title("Bar Plot")

axes[1] = plt.pie(df_labelled["Combined_Label"].value_counts(), 
                  labels=labels,
                  colors=[palette[key] for key in labels],
                  autopct='%1.2f%%',
                  shadow=False)
plt.title("Pie Plot")

for ax in [ax1]:
    for i in ax.containers:
        ax.bar_label(i)

fig.suptitle("Emission Label Distribution at Flavour Lab (AMS)", fontsize=18)
plt.show()